In [3]:
!sudo apt update
!sudo apt install -y python3-pip libnetcdf-dev libhdf5-dev

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease0m                  
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
108 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.5).
The following additional packages will be installed:
  hdf5-helpers libaec-dev libaec0 libcurl4-openssl-dev libgfortran5
  libhdf5-103-1 libhdf5-cpp-103-1 libhdf5-fortran-102 libhdf5-hl-100
  libhdf5-hl-cpp-100 libhdf5-hl-fortran-100 libjpeg-dev libjpeg-turbo8
  libjpeg-turbo8-dev libjpeg8 libjpeg8-dev lib

In [4]:
!pip install netCDF4 numpy torch torchvision matplotlib scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 18.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 9.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [5]:
import netCDF4
import numpy
import torch
import torchvision
import matplotlib
import scipy
print("netCDF4:", netCDF4.__version__)
print("numpy:", numpy.__version__)
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("matplotlib:", matplotlib.__version__)
print("scipy:", scipy.__version__)
print("CUDA disponible:", torch.cuda.is_available())

netCDF4: 1.7.2
numpy: 1.26.4
torch: 2.7.0+cu126
torchvision: 0.22.0+cu126
matplotlib: 3.10.1
scipy: 1.15.2
CUDA disponible: True


In [13]:
import os
import glob
import numpy as np
import netCDF4 as nc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
import matplotlib.pyplot as plt

# Configuración
data_dir = "/home/first_try_nc"
output_dir = "/home/model_output"
seq_len = 6  # Pasos de entrada
pred_len = 1  # Pasos de salida
batch_size = 1
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implementación personalizada de ConvLSTM2D
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias

        self.conv = nn.Conv3d(
            in_channels=input_dim + hidden_dim,
            out_channels=4 * hidden_dim,
            kernel_size=(kernel_size, kernel_size, kernel_size),
            padding=(self.padding, self.padding, self.padding),
            bias=bias
        )

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        print(f"Input tensor shape: {input_tensor.shape}, h_cur shape: {h_cur.shape}")  # Depuración
        combined = torch.cat([input_tensor, h_cur], dim=1)  # [batch, input_dim+hidden_dim, z, height, width]
        combined_conv = self.conv(combined)  # [batch, 4*hidden_dim, z, height, width]
        cc_i, cc_f, cc_c, cc_o = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        c_next = f * c_cur + i * torch.tanh(cc_c)
        o = torch.sigmoid(cc_o)
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        z, height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, z, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, z, height, width, device=self.conv.weight.device))

class ConvLSTM2D(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first=False, bias=True):
        super(ConvLSTM2D, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first

        self.cells = nn.ModuleList([
            ConvLSTMCell(
                input_dim if i == 0 else hidden_dim,
                hidden_dim,
                kernel_size,
                bias
            ) for i in range(num_layers)
        ])

    def forward(self, input_tensor, hidden_state=None):
        # input_tensor: (batch_size, seq_len, z, height, width, channels)
        if self.batch_first:
            input_tensor = input_tensor.permute(0, 1, 5, 2, 3, 4)  # (batch_size, seq_len, channels, z, height, width)
        else:
            input_tensor = input_tensor.permute(1, 0, 5, 2, 3, 4)

        batch_size, seq_len, channels, z, height, width = input_tensor.size()
        print(f"Input shape to ConvLSTM2D: {input_tensor.size()}")  # Depuración

        if hidden_state is None:
            hidden_state = [
                self.cells[l].init_hidden(batch_size, (z, height, width))
                for l in range(self.num_layers)
            ]

        cur_layer_input = input_tensor
        output_inner = []

        for t in range(seq_len):
            for layer_idx in range(self.num_layers):
                h, c = hidden_state[layer_idx]
                # Asegurar que cur_layer_input[:, t] tenga 5 dimensiones
                input_t = cur_layer_input[:, t]  # [batch, channels, z, height, width]
                if input_t.dim() == 4:  # Si channels=1 se elimina
                    input_t = input_t.unsqueeze(1)  # [batch, 1, z, height, width]
                h, c = self.cells[layer_idx](input_t, (h, c))
                hidden_state[layer_idx] = (h, c)
                cur_layer_input = h.unsqueeze(1) if layer_idx < self.num_layers - 1 else h
            output_inner.append(h)

        output = torch.stack(output_inner, dim=1 if self.batch_first else 0)
        return output, hidden_state

# Dataset
class RadarDataset(Dataset):
    def __init__(self, data_dir, seq_len=6, pred_len=1):
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.folders = sorted([f for f in glob.glob(os.path.join(data_dir, "*")) if os.path.isdir(f)])
        if not self.folders:
            raise ValueError(f"No se encontraron subcarpetas en {data_dir}")
        print(f"Subcarpetas encontradas: {len(self.folders)}")
        
        self.sequences = []
        for folder in self.folders:
            files = sorted(glob.glob(os.path.join(folder, "*.nc")))
            print(f"{folder}: {len(files)} NetCDFs")
            if len(files) >= seq_len + pred_len:
                for i in range(len(files) - seq_len - pred_len + 1):
                    self.sequences.append(files[i:i + seq_len + pred_len])
        if not self.sequences:
            raise ValueError(f"No se encontraron secuencias válidas (necesitan {seq_len + pred_len} NetCDFs por carpeta)")
        print(f"Total secuencias: {len(self.sequences)}")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq_files = self.sequences[idx]
        x = []
        for f in seq_files[:self.seq_len]:
            ds = nc.Dataset(f)
            dbz = ds.variables['DBZ'][:]  # Shape: (1, 18, 500, 500)
            ds.close()
            dbz = (dbz + 29) / (60 + 29)
            x.append(dbz[0])
        x = np.stack(x, axis=0)  # Shape: (6, 18, 500, 500)
        y = []
        for f in seq_files[self.seq_len:self.seq_len + pred_len]:
            ds = nc.Dataset(f)
            dbz = ds.variables['DBZ'][:]  # Shape: (1, 18, 500, 500)
            ds.close()
            dbz = (dbz + 29) / (60 + 29)
            y.append(dbz[0])
        y = np.stack(y, axis=0)  # Shape: (1, 18, 500, 500)
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)  # Shape: (6, 18, 500, 500, 1)
        y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)  # Shape: (1, 18, 500, 500, 1)
        print(f"x shape: {x_tensor.shape}, y shape: {y_tensor.shape}")  # Depuración
        return x_tensor, y_tensor

# Modelo ConvLSTM
class ConvLSTM(nn.Module):
    def __init__(self, in_channels=1, hidden_channels=32, num_layers=2, kernel_size=3):
        super(ConvLSTM, self).__init__()
        self.convlstm = ConvLSTM2D(
            input_dim=in_channels,
            hidden_dim=hidden_channels,
            kernel_size=kernel_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.conv = nn.Conv3d(hidden_channels, 1, kernel_size=(kernel_size, kernel_size, kernel_size),
                              padding=(1, 1, 1))

    def forward(self, x):
        print(f"Input to ConvLSTM: {x.shape}")  # Depuración
        x, _ = self.convlstm(x)  # Shape: (batch, seq_len, hidden_channels, z, height, width)
        x = x[:, -1]  # Último paso: (batch, hidden_channels, z, height, width)
        x = self.conv(x)  # Shape: (batch, 1, z, height, width)
        return x

# Crear directorio de salida
os.makedirs(output_dir, exist_ok=True)

# Cargar datos
try:
    dataset = RadarDataset(data_dir, seq_len, pred_len)
except ValueError as e:
    print(f"Error en dataset: {e}")
    exit(1)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Inicializar modelo
model = ConvLSTM(hidden_channels=32, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
scaler = GradScaler('cuda')

# Entrenamiento
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        print(f"Batch x shape: {x.shape}, y shape: {y.shape}")  # Depuración
        optimizer.zero_grad()
        with autocast('cuda'):
            y_pred = model(x)  # Shape: (batch, 1, z, height, width)
            loss = criterion(y_pred, y[:, 0])  # Shape: (batch, 1, z, height, width)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.6f}")

# Guardar modelo
torch.save(model.state_dict(), os.path.join(output_dir, "convlstm.pth"))

# Generar predicciones (ejemplo)
model.eval()
with torch.no_grad():
    x, y = dataset[0]
    x = x.unsqueeze(0).to(device)  # Shape: (1, 6, 18, 500, 500, 1)
    y_pred = model(x)  # Shape: (1, 1, 18, 500, 500)
    y_pred = y_pred.cpu().numpy() * (60 + 29) - 29
    y = y.numpy() * (60 + 29) - 29
    ds_out = nc.Dataset(os.path.join(output_dir, "pred_0.nc"), 'w', format='NETCDF4')
    ds_out.createDimension('time', 1)
    ds_out.createDimension('z', 18)
    ds_out.createDimension('y', 500)
    ds_out.createDimension('x', 500)
    dbz_var = ds_out.createVariable('DBZ', 'f4', ('time', 'z', 'y', 'x'))
    dbz_var[:] = y_pred
    ds_out.close()

Subcarpetas encontradas: 10
/home/first_try_nc/2010010119: 7 NetCDFs
/home/first_try_nc/201010232: 7 NetCDFs
/home/first_try_nc/2010102321: 7 NetCDFs
/home/first_try_nc/2010102336: 7 NetCDFs
/home/first_try_nc/2011111522: 7 NetCDFs
/home/first_try_nc/201111154: 7 NetCDFs
/home/first_try_nc/2012111615: 7 NetCDFs
/home/first_try_nc/2012111620: 7 NetCDFs
/home/first_try_nc/2012111625: 7 NetCDFs
/home/first_try_nc/201211167: 7 NetCDFs
Total secuencias: 10
x shape: torch.Size([6, 18, 500, 500, 1]), y shape: torch.Size([1, 18, 500, 500, 1])
Batch x shape: torch.Size([1, 6, 18, 500, 500, 1]), y shape: torch.Size([1, 1, 18, 500, 500, 1])
Input to ConvLSTM: torch.Size([1, 6, 18, 500, 500, 1])
Input shape to ConvLSTM2D: torch.Size([1, 6, 1, 18, 500, 500])
Input tensor shape: torch.Size([1, 1, 18, 500, 500]), h_cur shape: torch.Size([1, 32, 18, 500, 500])
Input tensor shape: torch.Size([1, 32, 18, 500, 500]), h_cur shape: torch.Size([1, 32, 18, 500, 500])
Input tensor shape: torch.Size([1, 1, 18,

OutOfMemoryError: CUDA out of memory. Tried to allocate 550.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 504.69 MiB is free. Process 2950297 has 23.19 GiB memory in use. Of the allocated memory 21.78 GiB is allocated by PyTorch, and 1.11 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)